# Pydantic


In [ ]:
%pip install -q "pydantic>=2.12.3,<3" "sqlalchemy>=2.0.44,<3"
import pydantic, sqlalchemy, sys
print("pydantic", pydantic.__version__, "| sqlalchemy", sqlalchemy.__version__, "| python", sys.version.split()[0])


## План
- Настройки моделей: иммутабельность и поведение `extra`
- Именованные модели, пространства имён, защищённые префиксы
- ORM-режим: `from_attributes`
- Расширенные поля: вычисляемые, дискриминаторы, приватные
- Валидации: алиасы, ограничения, валидаторы, `Annotated`-валидаторы
- Декораторы Pydantic


## 1. Настройки и конфигурация моделей
Покажем иммутабельность (`frozen`) и разные режимы работы с лишними полями: `allow`, `forbid`, `ignore`.


In [ ]:
from pydantic import BaseModel, ConfigDict

# Иммутабельная модель
class ImmutableGeoPoint(BaseModel):
    model_config = ConfigDict(frozen=True)
    lat: float
    lon: float

pt = ImmutableGeoPoint(lat=1.0, lon=2.0)
print("Immutable:", pt)
try:
    pt.lat = 10
except Exception as e:
    print("Attempt to mutate:", type(e).__name__, str(e)[:80])

In [ ]:

class FlexibleGeoPoint(BaseModel, extra='allow'):
    lat: float
    lon: float

flex = FlexibleGeoPoint(lat=1.0, lon=2.0, name='Kurgan')
print("Allow extra, name:", getattr(flex, 'name', None))

In [ ]:

class StrictGeoPoint(BaseModel, extra='forbid'):
    lat: float
    lon: float

try:
    StrictGeoPoint(lat=1.0, lon=2.0, name='Kurgan')
except Exception as e:
    print("Forbid extra:", type(e).__name__, str(e).split('\n')[0])

In [ ]:

class IgnoreExtraGeoPoint(BaseModel, extra='ignore'):
    lat: float
    lon: float

ig = IgnoreExtraGeoPoint(lat=1.0, lon=2.0, name='Kurgan')
print("Ignore extra has name?", hasattr(ig, 'name'))


## 2. Пространства имен
Покажем защиту служебных префиксов `protected_namespaces` и последствия их конфликта с полями модели.


In [ ]:
import warnings
from pydantic import BaseModel, ConfigDict

warnings.filterwarnings('error')

try:
    class ModelWithProtected(BaseModel):
        model_prefixed_field: str
        also_protect_field: str
        model_config = ConfigDict(
            protected_namespaces=('protect_me_', 'also_protect_')
        )
except UserWarning as e:
    print("Protected namespaces warning -> error:", e)
finally:
    warnings.resetwarnings()


## 3. ORM-режим и `from_attributes`

In [ ]:
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base
from pydantic import BaseModel, ConfigDict, Field, constr

Base = declarative_base()

class StudentOrm(Base):
    __tablename__ = 'students'
    id = sa.Column('id', sa.Integer, primary_key=True)
    name = sa.Column('name', sa.String(20))

class StudentModel(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    id: int
    name: constr(max_length=20)

student_orm = StudentOrm(id=1, name='Ivan')
student_model = StudentModel.model_validate(student_orm)
print(student_model)

In [ ]:

# Reserved names
class MyModel(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    metadata: dict[str, str] = Field(alias='metadata_')

class SQLModel(Base):
    __tablename__ = 'my_table'
    id = sa.Column('id', sa.Integer, primary_key=True)
    metadata_ = sa.Column('metadata', sa.JSON)

sql_model = SQLModel(metadata_={'key': 'val'}, id=2)
print(MyModel.model_validate(sql_model).model_dump())
print(MyModel.model_validate(sql_model).model_dump(by_alias=True))


## 4. Расширенные поля: вычисляемые, дискриминаторы, приватные


In [ ]:
from functools import cached_property
import random
from pydantic import BaseModel, computed_field

class Square(BaseModel):
    width: float

    @computed_field
    @property
    def area(self) -> float:
        return self.width ** 2

    @area.setter
    def area(self, new_area: float) -> None:
        self.width = new_area ** 0.5

    @computed_field(alias='the magic number', repr=False)
    @cached_property
    def random_number(self) -> int:
        return random.randint(0, 1000)

sq = Square(width=2.0)
print(repr(sq))

print(sq.random_number)

print(sq.model_dump())

sq.area = 1.69

print(sq.model_dump_json(by_alias=True))


### Кэширование computed_field
`computed_field` с `@cached_property` вычисляется один раз и кэшируется в инстансе. Изменение других полей не пересчитывает значение автоматически; нужно сбросить кэш (удалить атрибут) или создать новый инстанс.


In [ ]:
# Демонстрация кэширования random_number и пересчёта area
sq = Square(width=2.0)

first = sq.random_number
second = sq.random_number
print("Первый вызов:", first)
print("Второй вызов (кэш):", second)
print("Совпадают?", first == second)

# Меняем ширину — area пересчитается, но random_number останется кэшированным
sq.width = 5.0
print("area после изменения width:", sq.area)
print("random_number после изменения width (кэш):", sq.random_number)
print("Рандом всё ещё равен первому?", sq.random_number == first)

# Явная инвалидация кэша для random_number
if 'random_number' in sq.__dict__:
    del sq.__dict__['random_number']
third = sq.random_number
print("После инвалидации кэша, новый random_number:", third)
print("Изменился ли random_number?", third != first)

# Пересоздание модели создаёт новый кэш
sq2 = sq.model_copy(update={'width': 1.0})
print("Новый инстанс random_number отличается?", sq2.random_number != sq.random_number)


In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class Cat(BaseModel):
    pet_type: Literal['cat']
    age: int

class Dog(BaseModel):
    pet_type: Literal['dog']
    age: int

class PetModel(BaseModel):
    pet: Cat | Dog = Field(discriminator='pet_type')

print(PetModel.model_validate({'pet': {'pet_type': 'cat', 'age': 12}}))


In [ ]:
from typing import Any
from pydantic import BaseModel, PrivateAttr, ConfigDict

class Cache(BaseModel):
    model_config = ConfigDict(validate_assignment=True)
    data: dict[str, Any]
    _hits: int = PrivateAttr(0)

    def get(self, key: str) -> Any:
        self._hits += 1
        return self.data.get(key)

c = Cache(data={"a": 2, "b": 3})
print(c.get("a"))
print("hits:", c._hits)
print(c.model_dump())


## 5. Валидация и алиасы


In [ ]:
import uuid
from pydantic import BaseModel, Field, ConfigDict, AliasGenerator
from pydantic.alias_generators import to_camel

student_dict = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС'
}

class AliasedStudent(BaseModel):
    id: uuid.UUID = Field(alias='id')
    name: str = Field(alias='name')
    average_score: float = Field(alias='averageScore')
    department: str = Field(alias='department')

print(AliasedStudent(**student_dict))

In [ ]:

class AliasesByConfig(BaseModel):
    model_config = ConfigDict(
        validate_by_name=True,
        validate_by_alias=True,
        alias_generator=AliasGenerator(
            validation_alias=to_camel,
            serialization_alias=to_camel,
        )
    )
    id: uuid.UUID
    name: str
    average_score: float
    department: str

inst = AliasesByConfig(**student_dict)
print(inst)
print(inst.model_dump(by_alias=True))


### Ограничения полей (constrained types)

In [ ]:
import uuid
from pydantic import BaseModel, ConfigDict, AliasGenerator, confloat
from pydantic.alias_generators import to_camel

ok = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС'
}

bad = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 5.2,  # or less than 2.0
    'department': 'ПОАС'
}

class StudentWithConstraints(BaseModel):
    model_config = ConfigDict(
        validate_by_name=True,
        validate_by_alias=True,
        alias_generator=AliasGenerator(
            validation_alias=to_camel,
            serialization_alias=to_camel,
        )
    )
    id: uuid.UUID
    name: str
    average_score: confloat(ge=2, le=5)
    department: str

print(StudentWithConstraints(**ok))
try:
    print(StudentWithConstraints(**bad))
except Exception as e:
    print("Validation error:", type(e).__name__, str(e).split('\n')[0])


### Кастомные валидаторы полей


In [ ]:
import datetime
import uuid
from datetime import timedelta
from pydantic import BaseModel, ConfigDict, AliasGenerator, confloat, field_validator
from pydantic.alias_generators import to_camel

ok = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС',
    'dateBirth': '2000-01-01'
}

too_young = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС',
    'dateBirth': '2010-01-01'
}

class StudentWithAge(BaseModel):
    model_config = ConfigDict(
        validate_by_name=True,
        validate_by_alias=True,
        alias_generator=AliasGenerator(
            validation_alias=to_camel,
            serialization_alias=to_camel,
        )
    )
    id: uuid.UUID
    name: str
    average_score: confloat(ge=2, le=5)
    department: str
    date_birth: datetime.date

    @field_validator('date_birth')
    def ensure_18_or_over(cls, value):
        eighteen_years_ago = (datetime.datetime.now() - timedelta(days=365*18)).date()
        if value > eighteen_years_ago:
            raise ValueError('Too young')
        return value

print(StudentWithAge(**ok))
try:
    print(StudentWithAge(**too_young))
except Exception as e:
    print("Validation error:", type(e).__name__, str(e).split('\n')[0])


### Enum в качестве типа поля
Покажем ограничение допускаемых значений через `StrEnum`.


In [ ]:
import uuid
import datetime
from enum import StrEnum
from datetime import timedelta
from pydantic import BaseModel, ConfigDict, AliasGenerator, confloat, field_validator
from pydantic.alias_generators import to_camel

class Department(StrEnum):
    POAS = 'ПОАС'

ok = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС',
    'dateBirth': '2000-01-01'
}

bad_dep = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'БИАС',
    'dateBirth': '2000-01-01'
}

class StudentWithEnum(BaseModel):
    model_config = ConfigDict(
        validate_by_name=True,
        validate_by_alias=True,
        alias_generator=AliasGenerator(
            validation_alias=to_camel,
            serialization_alias=to_camel,
        )
    )
    id: uuid.UUID
    name: str
    average_score: confloat(ge=2, le=5)
    department: Department
    date_birth: datetime.date

    @field_validator('date_birth')
    def ensure_18_or_over(cls, value):
        eighteen_years_ago = (datetime.datetime.now() - timedelta(days=365*18)).date()
        if value > eighteen_years_ago:
            raise ValueError('Too young')
        return value

print(StudentWithEnum(**ok))
try:
    print(StudentWithEnum(**bad_dep))
except Exception as e:
    print("Validation error:", type(e).__name__, str(e).split('\n')[0])


### Annotated‑валидаторы


In [ ]:
import uuid
import datetime
from typing import Annotated
from datetime import timedelta
from pydantic import BaseModel, ConfigDict, AliasGenerator, confloat, field_validator, AfterValidator
from pydantic.alias_generators import to_camel


def validate_phone(v: str) -> str:
    if not v.startswith('+7'):
        raise ValueError('Phone should start with +7')
    return v

Phone = Annotated[str, AfterValidator(validate_phone)]

ok = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС',
    'dateBirth': '2000-01-01',
    'phone': '+7900000000'
}

bad_phone = {
    'id': str(uuid.uuid4()),
    'name': 'Ivan',
    'averageScore': 3.5,
    'department': 'ПОАС',
    'dateBirth': '2000-01-01',
    'phone': '8900000000'
}

class StudentWithPhone(BaseModel):
    model_config = ConfigDict(
        validate_by_name=True,
        validate_by_alias=True,
        alias_generator=AliasGenerator(
            validation_alias=to_camel,
            serialization_alias=to_camel,
        )
    )
    id: uuid.UUID
    name: str
    average_score: confloat(ge=2, le=5)
    department: str
    date_birth: datetime.date
    phone: Phone

    @field_validator('date_birth')
    def ensure_18_or_over(cls, value):
        eighteen_years_ago = (datetime.datetime.now() - timedelta(days=365*18)).date()
        if value > eighteen_years_ago:
            raise ValueError('Too young')
        return value

print(StudentWithPhone(**ok))
try:
    print(StudentWithPhone(**bad_phone))
except Exception as e:
    print("Validation error:", type(e).__name__, str(e).split('\n')[0])


### Автоматический каст типов


In [ ]:
from pydantic import BaseModel

class GeoPoint(BaseModel):
    lat: float
    lon: float

point = {"lat": "12.345", "lon": "12.345"}
geo_point = GeoPoint(**point)
print(type(geo_point.lat), geo_point.lat)
print(type(geo_point.lon), geo_point.lon)


## 6. Декораторы Pydantic: `@validate_call`
Валидация аргументов функций и уточнение ограничений через `Annotated`.


In [ ]:
from typing import Annotated
from pydantic import validate_call, StringConstraints

@validate_call
def greet_user(name: str, greeting: str = 'Hello') -> str:
    return f'{greeting}, {name}!'

print(greet_user('Ivan'))
try:
    print(greet_user(123))  # type: ignore
except Exception as e:
    print('validate_call error:', type(e).__name__, str(e).split('\n')[0])

@validate_call
def greet_user_new(name: Annotated[str, StringConstraints(min_length=10)], greeting: str = 'Hello') -> str:
    return f'{greeting}, {name}!'

try:
    print(greet_user_new('Ivan'))
except Exception as e:
    print('Annotated constraint error:', type(e).__name__, str(e).split('\n')[0])


## Ссылка на документацию
- Документация: https://docs.pydantic.dev/latest/
